In [119]:
from pyspark.sql import SparkSession, SQLContext
import os
import io
# to download data
import wget
# storing and analysing data
import pandas as pd
from pyspark.sql.types import StructType,StructField, StringType, IntegerType


class DownloadData():
    
    def __init__(self, file, folder_csv, folder_source):
        self.file          = file
        self.folder_csv    = folder_csv
        self.folder_source = folder_source

    def delete_file(self):
        if len(os.listdir(self.folder_csv)) > 0:
            for file in os.listdir(self.folder_csv):
                if file[0] != ".":
                    os.remove(os.path.join(self.folder_csv, file))
        return "Delete File"
        

    def download_file(self):
        # download files
        for url in self.file:
            filename = wget.download(url)
            os.replace(folder_source+filename, folder_csv+filename)
        return "Download file"



class TransformFile(DownloadData):


    def __init__(self,spark_session, folder_parquet,folder_csv=None, folder_source=None, file=None):
        super().__init__(file, folder_csv, folder_source)
        self.spark_session   = spark_session
        self.folder_parquet  = folder_parquet


    def csv_to_parquet(self):
        try:
            if len(os.listdir(folder_csv)) > 0:
                for file in os.listdir(folder_csv):
                    if file[0] != ".":
                        print(file)
                        pandas_df = pd.read_csv(folder_csv + file)
                        # Convierte el DataFrame de pandas a un DataFrame de Spark
                        spark_df = self.spark_session.createDataFrame(pandas_df)
                        # Llena los valores nulos con un valor predeterminado (ajusta según tus necesidades)
                        spark_df = spark_df.fillna("Null")
                        # Guarda el DataFrame como un archivo Parquet en el directorio especificado
                        spark_df.write.mode("overwrite").parquet(folder_parquet)
                        
            return "Transform"
        except Exception as e:
            return f"Error {e}"



class ChangeColumnType():

    def __init__(self, folder_parquet):
        self.folder_parquet = folder_parquet
           

    def update_parquet(self):
        try:
            for file in os.listdir(self.folder_parquet):
                if file[0] != "." and file[0] != "_":
                    df_parquet = pd.read_parquet(self.folder_parquet+file)
                    columns_int = df_parquet.select_dtypes(include=['int', 'float']).columns
                    df_parquet[columns_int] = df_parquet[columns_int].astype(object)
                    df_parquet.to_parquet(self.folder_parquet+file, index=True, compression="snappy", engine='auto')
                    df_parquet = pd.read_parquet(self.folder_parquet+file)
            return df_parquet.dtypes
        except Exception as e:
            return f"Error al leer el archivo Parquet: {e}"

                



In [152]:
if __name__ == "__main__":

    #Spark Session
    spark = SparkSession.builder.appName("N5ProcesadorDeDatos").getOrCreate()

    #Url from process
    data_time_series      = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/"
    data_iso_fips         = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/"
    data_situation_report = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/who_covid_19_situation_reports/who_covid_19_sit_rep_time_series/"

    #Path folders for files
    folder_source = ""
    folder_csv = ""
    folder_parquet = ""

    #File to Process
    urls = [data_time_series+'time_series_covid19_confirmed_global.csv', 
            data_time_series+'time_series_covid19_deaths_global.csv',
            data_time_series+'time_series_covid19_recovered_global.csv',
            data_iso_fips+'UID_ISO_FIPS_LookUp_Table.csv',
            data_situation_report+'who_covid_19_sit_rep_time_series.csv'
            ]
    
    #Download Data from Github
    data = DownloadData( urls, folder_csv, folder_source)
    delete_file = data.delete_file()
    get_data = data.download_file()

    #Process file csv to parquet
    process = TransformFile(spark ,folder_parquet)
    process_file = process.csv_to_parquet()
    print( process_file)

    #Update parquet
    update = ChangeColumnType(folder_parquet)
    updtae_column = update.update_parquet()
    

       

/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


100% [............................................................................] 159506 / 159506time_series_covid19_confirmed_global.csv


/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:
23/12/03 22:32:56 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
23/12/03 22:32:56 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
23/12/03 22:32:56 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
23/12/03 22:32:56 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
23/12/03 22:32:56 WARN MemoryManager: Total allocation exceeds 95,00% 

who_covid_19_sit_rep_time_series.csv
time_series_covid19_recovered_global.csv


/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:
23/12/03 22:32:59 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
23/12/03 22:32:59 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
23/12/03 22:32:59 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
23/12/03 22:32:59 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
23/12/03 22:32:59 WARN MemoryManager: Total allocation exceeds 95,00% 

UID_ISO_FIPS_LookUp_Table.csv
time_series_covid19_deaths_global.csv


/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


Transform
